# Representational Similarity Analysis

This notebook contains code to reproduce our representational similarity analysis (RSA) for both experimental conditions (sentence condition and picture condition) and the three brain networks (left-hemisphere language netowrk, right-hemisphere language network, and visual network).

In [ ]:
# importing necessary libraries
from src.paths import ROOT
from scipy.stats import spearmanr
from src.utils import open_json, dict_to_json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
# from src.best_layers import *

In [ ]:
# importing dict with brain RDMs
brain_rdms = open_json(ROOT / 'data/participant_rdms.json')

## Sentence condition

In [ ]:
# importing model rmds for the sentence condition
model_rdms = open_json(ROOT / 'data/sentences_model_rdms.json')

In [ ]:
triu_indices = np.triu_indices(n=180, m=180, k=1)

# creating a dict to store results
rsa_results_sentence = {}

for model in model_rdms:
    
    # creating a sub-dict for each model to store layer-specific correlations
    rsa_results_sentence[model] = {}
    n_layers = len(model_rdms[model])

    for network in ['visual', 'languageLH', 'languageRH']:
        rsa_results_sentence[model][network] = {'rho':[], 'pval':[]}
        
        # selecting the off-diagonal of the brain rdm 
        brain_rdm = np.array(brain_rdms['sentence'][network])[triu_indices]
        
        if n_layers > 1:

            # iterating through layers
            for layer in range(n_layers):

                # selecting the off-diagonal of the model rdm 
                layer_rdm = np.array(model_rdms[model][f"layer_{layer+1}"])[triu_indices]

                # computing rsa (Spearman correlation)
                rho, pval = spearmanr(layer_rdm, brain_rdm)
                
                # storing result in the dict
                rsa_results_sentence[model][network]['rho'].append(rho.item())
                rsa_results_sentence[model][network]['pval'].append(pval.item())

        else:

            for layer in model_rdms[model]:

                # selecting the off-diagonal of the model rdm 
                model_rdm = np.array(model_rdms[model][layer])[triu_indices]

                # computing rsa (Spearman correlation)
                rho, pval = spearmanr(model_rdm, brain_rdm)

                # storing result in the dict
                rsa_results_sentence[model][network]['rho'].append(rho.item())
                rsa_results_sentence[model][network]['pval'].append(pval.item())

In [ ]:
# saving result dict as a json
dict_to_json(rsa_results_sentence, 'results/rsa_sentence.json')

## Picture

In [ ]:
# importing model rmds for the picture condition
model_rdms = open_json(ROOT / 'data/picture_model_rdms.json')

In [ ]:
triu_indices = np.triu_indices(n=180, m=180, k=1)

# creating a dict to store results
rsa_results_picture = {}

for model in model_rdms:

    # creating a sub-dict for each model to store layer-specific correlations
    rsa_results_picture[model] = {}
    n_layers = len(model_rdms[model])

    for network in ['visual', 'languageLH', 'languageRH']:
        rsa_results_picture[model][network] = {'rho':[], 'pval':[]}

        # selecting the off-diagonal of the brain rdm 
        brain_rdm = np.array(brain_rdms['picture'][network])[triu_indices]
        
        if n_layers > 1:
            # iterating through model layers
            for layer in range(n_layers):

                # selecting the off-diagonal of the model rdm 
                layer_rdm = np.array(model_rdms[model][f"layer_{layer+1}"])[triu_indices]

                # computing rsa (Spearman correlation)
                rho, pval = spearmanr(layer_rdm, brain_rdm)
                
                # storing result in the dict
                rsa_results_picture[model][network]['rho'].append(rho.item())
                rsa_results_picture[model][network]['pval'].append(pval.item())

        else:

            for layer in model_rdms[model]:

                # selecting the off-diagonal of the model rdm 
                model_rdm = np.array(model_rdms[model][layer])[triu_indices]

                # computing rsa (Spearman correlation)
                rho, pval = spearmanr(model_rdm, brain_rdm)

                # storing result in the dict
                rsa_results_picture[model][network]['rho'].append(rho.item())
                rsa_results_picture[model][network]['pval'].append(pval.item())

In [ ]:
# saving result dict as a json
dict_to_json(rsa_results_picture, 'results/rsa_picture.json')

It's a good idea to print the best layers for each network and experimental condition and save them in `src/best_layers.py`, so that they can be easily imported within other modules.

In [ ]:
# saving best layers
best_layers_sentence = {}

networks = ['languageLH', 'languageRH', 'visual']

for model in ['clip', 'align', 'lxmert', 'visualbert', 'llava', 'llama', 'idefics2', 'mistral', 'bert']:
    best_layers_sentence[model]  = {}
    for network in networks:
        best_layers_sentence[model][network] = f"layer_{np.array(rsa_results_sentence[model][network]['rho']).argmax()+1}"


In [63]:
# saving best layers
best_layers_picture = {}

for model in ['lxmert', 'visualbert', 'llava', 'llama', 'idefics2', 'mistral', 'bert']:
    best_layers_picture[model]  = {}
    for network in networks:
        best_layers_picture[model][network] = f"layer_{np.array(rsa_results_picture[model][network]['rho']).argmax()+1}"


## Statistical Tests

This section of the notebook contains code to reproduce the statistical tests used to assess:

1. whether differences in models' brain alignment are statistically significant or not
2. whether the most aligned model layer is signicantly more brain-aligned than the other model layers

In [ ]:
# reimporting RSA results (if necessary)

rsa_results_picture = open_json('results/rsa_picture.json')
rsa_results_sentence = open_json('results/rsa_sentence.json')

### Tests for the differences between models

Here, we want to test whether the brain-alignment yielded by each model's best layer is statististically significantly different from the brain alignment produced by other models' best layers. 

In [ ]:
# creating a dict containing only the best-layer RSA value for each model (sentence condition)

sentence_values = {}

models = ['clip', 'align', 'lxmert', 'visualbert', 'idefics2', 'llava', 'mistral', 'llama', 'bert', 'glove']
networks = ['languageLH', 'languageRH', 'visual']

for network in networks:
    sentence_values[network] = {}
    for i, (model) in enumerate(models):
        model_values = np.array(rsa_results_sentence[model][network]['rho'])
        sentence_values[network][model] = model_values.max().item()

In [ ]:
# creating a dict containing only the best-layer RSA value for each model (picture condition)

picture_values = {}
models = ['clip', 'align', 'lxmert', 'visualbert', 'idefics2', 'llava', 'mistral', 'llama', 'bert', 'glove']
networks = ['languageLH', 'languageRH', 'visual']

for network in networks:
    picture_values[network] = {}
    for i, (model) in enumerate(models):
        model_values = np.array(rsa_results_picture[model][network]['rho'])
        picture_values[network][model] = model_values.max().item()

In [ ]:
# importing thif function for computing statistical significance
from src.statistical_tests import corr_significance_test

In [ ]:
networks = ['languageLH', 'languageRH', 'visual']
models = ['clip', 'align', 'lxmert', 'visualbert', 'idefics2', 'llava', 'mistral', 'llama', 'bert', 'glove']
n = 180
size = (n*n-n)/2
indices = np.triu_indices(n=len(models), m=len(models), k=1)

for ind1, ind2 in zip(indices[0], indices[1]):
    
    pair_values_sent = []
    pair_values_pic = []

    # iterating through brain networks
    for network in networks:
        
        # calculating pvalues for the sentence condition
        corr1 = sentence_values[network][models[ind1]]
        corr2 = sentence_values[network][models[ind2]]
        sent_p_val = corr_significance_test(size, corr1=corr1, corr2=corr2)
        pair_values_sent.append(round(sent_p_val, 3))
        
        # calculating pvalues for the picture condition
        corr1 = picture_values[network][models[ind1]]
        corr2 = picture_values[network][models[ind2]]
        pic_p_val = corr_significance_test(size, corr1=corr1, corr2=corr2)
        pair_values_pic.append(round(pic_p_val, 3))
        
    # printing p-values in a latex-friendly format
    print(f"{[models[ind1]]} - {[models[ind2]]} & {pair_values_sent[0]} & {pair_values_pic[0]} & {pair_values_sent[1]} & {pair_values_pic[1]} & {pair_values_sent[2]} & {pair_values_pic[2]}  \\\\")
        

            

### Tests for best layer's significance

Here, we check whether models' best layer (i.e., the most brain-aligned) is statistically signifcantly more aligned than the other layers.

In [ ]:
def get_best_layer_significance(results_dict, model, brain_network):
    
    '''
        This function takes as input a dict storing rsa results (e.g., rsa_results_sentence),
        the string indicating a model (which should be a key in results_dict) and the string
        indicating a brain network (i.e., 'languageLH', 'languageRH', 'visual'). The function
        computes Bonferroni-corrected p-values for the differences between each layer and the 
        best layer and prints 'True' or 'False' depending on whether the difference is 
        statistically significant or not.

    '''    
    
    
    rsa_vals = np.array(results_dict[model][brain_network]['rho']) # selecting rsa values for the model of interest
    
    # obtaining index of the best layer and its rsa value
    argmax = rsa_vals.argmax().item()
    val_max = rsa_vals.max().item()
    
    n_compar = rsa_vals.shape[0]-1 # defining the number of comparisons as the number of layers - 1 (we don't compare the best layer with itself)

    # iterating through layers
    for layer in range(rsa_vals.shape[0]):

        # skipping the best layer
        if layer != argmax:

            # computing p-value
            pval = corr_significance_test(16110, corr1=val_max, corr2=rsa_vals[layer])

            # printing significance of Bonferroni-corrected p-value
            print(layer+1, pval<0.05/n_compar)



In [ ]:
# example of function use
get_best_layer_significance(rsa_results_sentence, 'clip', 'languageLH')

## Picture condition ablation study

Here, we recompute RSA for the picture condition in the ablation study where we passed the same input (i.e., only the concept word) to both VLMs and language-only models.

In [ ]:
# importing relevant RDMs
vlm_rdms = open_json(ROOT / "data/concept_only_vlm_rdms.json")
lm_rdms = open_json(ROOT / "data/picture_model_rdms.json")

In [52]:
vlms = ['clip', 'align', 'lxmert', 'visualbert', 'idefics2', 'llava']
lms = ['bert', 'mistral', 'llama', 'glove']

In [ ]:
# creating a dict to store vlm results
rsa_dict = {}
triu_indices = np.triu_indices(n=180, m=180, k=1)


for model in vlms:

    # creating a dict for every model
    rsa_dict[model] = {}

    # iterating through networks
    for network in brain_rdms['picture']:

        # creating RDM
        brain_rdm = np.array(brain_rdms['picture'][network])[triu_indices]
        rsa_dict[model][network] = {'rho': [], 'pval': []}
        
        # iterating through layers
        for layer in vlm_rdms[model]:
            model_rdm = np.array(vlm_rdms[model][layer])[triu_indices]

            # computing RSA
            rho, pval = spearmanr(model_rdm, brain_rdm) 

            # storing results in dict
            rsa_dict[model][network]['rho'].append(rho.item())
            rsa_dict[model][network]['pval'].append(pval.item())

In [ ]:
# adding to the same dict the best-layer RSA values for the language-only models (we already have these 
# values stored, so no need to recompute them)

for model in lms:

    rsa_dict[model] = {}

    # iterating through networks
    for network in brain_rdms['picture']:

        brain_rdm = np.array(brain_rdms['picture'][network])[triu_indices]
        rsa_dict[model][network] = {'rho': [], 'pval': []}

        # iterating through layers
        for layer in lm_rdms[model]:
            model_rdm = np.array(lm_rdms[model][layer])[triu_indices]
            rho, pval = spearmanr(model_rdm, brain_rdm) 
            rsa_dict[model][network]['rho'].append(rho.item())
            rsa_dict[model][network]['pval'].append(pval.item())

### Statistical tests

As for the main experiment, here we have to test whether differences between models' RSA values are statistically significant. 

In [ ]:
networks = ['languageLH', 'languageRH', 'visual']
models = ['clip', 'align', 'lxmert', 'visualbert', 'idefics2', 'llava', 'mistral', 'llama', 'bert', 'glove']

n = 180
size = (n*n-n)/2
indices = np.triu_indices(n=len(models), m=len(models), k=1)

for network in networks:
    print('\n', network)
    
    for ind1, ind2 in zip(indices[0], indices[1]):

        corr1 = np.array(rsa_dict[models[ind1]][network]['rho']).max()
        corr2 = np.array(rsa_dict[models[ind2]][network]['rho']).max()
        p_val = corr_significance_test(size, corr1=corr1, corr2=corr2)

        # Bonferroni correction
        corrected_pval = 0.05 / len(indices[0])
        print(models[ind1], models[ind2], round(corrected_pval, 3))
        